# Starting EDA of pH project

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [37]:
csv_file = "output_file.csv"
df = pd.read_csv(csv_file)
df.head()

,Sequence,Count
0,AGTGCCATCGTGCGTATCCTTCACTCCTTGCTCGACAAGAAGCGAA...,91
1,AGTGCCATCGTGCGTATCCTTCACGTTTGTTCTACAAAAGCGAAGC...,86
2,AGTGCCATCGTGCGTATCCTGAACATTGACGTGTAGTCAATGCGAA...,83
3,AGTGCCATCGTGCGTATCCCGCTCCGCTAATATCTATGTTAGCGAA...,80
4,AGTGCCATCGTGCGTATCCTGAACATAGACGTTTAGTCTATGCGAA...,78


In [38]:
df.shape

(10383, 2)

In [39]:
sum(df.iloc[:, 0].isnull())

0

In [40]:
sum(df.iloc[:, 1].isnull())

0

In [41]:
df.dtypes

Sequence    object
Count        int64
dtype: object

In [42]:
# check base type proportions
def base_count(text):
    return dict(Counter(text))   

In [43]:
df['Base Counts'] = df['Sequence'].apply(base_count)
# df['Base Counts'][1]['A']
df['A'] = df['Base Counts'].astype(object).apply(lambda x: x.get('A', np.nan))
df['C'] = df['Base Counts'].astype(object).apply(lambda x: x.get('C', np.nan))
df['T'] = df['Base Counts'].astype(object).apply(lambda x: x.get('G', np.nan))
df['G'] = df['Base Counts'].astype(object).apply(lambda x: x.get('T', np.nan))

In [44]:
df.head().iloc[:, 3:]

,A,C,T,G
0,24,26,26,22
1,22,27,29,20
2,23,23,29,23
3,24,25,28,21
4,24,21,30,23


In [45]:
df['Length'] = df['Sequence'].apply(len)

In [46]:
sum((np.sum(df[['A', 'C', 'T', 'G']], axis=1) == df['Length']).isnull()) # Checks base counts add up to length of sequence

0

In [47]:
allowed_keys = {'A', 'C', 'T', 'G'} # Checks that there are no weird nonstandard bases
df['Nonstandard base'] = df['Base Counts'].astype(object).apply(lambda x: not set(x.keys()).issubset(allowed_keys))
np.sum(df['Nonstandard base'])

0

In [48]:
df.head()

,Sequence,Count,Base Counts,A,C,T,G,Length,Nonstandard base
0,AGTGCCATCGTGCGTATCCTTCACTCCTTGCTCGACAAGAAGCGAA...,91,"{'A': 24, 'G': 26, 'T': 22, 'C': 26}",24,26,26,22,98,False
1,AGTGCCATCGTGCGTATCCTTCACGTTTGTTCTACAAAAGCGAAGC...,86,"{'A': 22, 'G': 29, 'T': 20, 'C': 27}",22,27,29,20,98,False
2,AGTGCCATCGTGCGTATCCTGAACATTGACGTGTAGTCAATGCGAA...,83,"{'A': 23, 'G': 29, 'T': 23, 'C': 23}",23,23,29,23,98,False
3,AGTGCCATCGTGCGTATCCCGCTCCGCTAATATCTATGTTAGCGAA...,80,"{'A': 24, 'G': 28, 'T': 21, 'C': 25}",24,25,28,21,98,False
4,AGTGCCATCGTGCGTATCCTGAACATAGACGTTTAGTCTATGCGAA...,78,"{'A': 24, 'G': 30, 'T': 23, 'C': 21}",24,21,30,23,98,False


In [56]:
df['Length'].value_counts().sort_values(ascending=False)

Length
98     9893
97      325
99       65
96       42
95       11
94        7
88        7
87        4
91        4
84        3
90        3
93        3
86        3
85        3
92        3
89        1
107       1
102       1
108       1
82        1
100       1
83        1
Name: count, dtype: int64

The sequences range in length from 82 to 108, but most commonly at 98.
As I recally, NGS clips the first few base pairs down from nominal 100 because of poor quality.
The other variants in sequence length are likely due to mutations from PCR errors.